In [48]:
!pip install wikipedia

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=7084df376b9901859258a7e7052cd7bbd9ec14d6a63e667239ee405314931911
  Stored in directory: /home/soma/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [45]:
import pandas as pd
from wikidata.client import Client
import wikipedia

import requests
from requests import utils
from bs4 import BeautifulSoup

client = Client()

In [3]:
triplets = pd.read_json('data/poet_triplets.json')
triplets.head()

,source,edge,destination
0,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P31', 'name': 'osztály, amely...","{'wikidata_id': 'Q5', 'name': 'ember', 'descri..."
1,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P21', 'name': 'nem', 'descrip...","{'wikidata_id': 'Q6581097', 'name': 'férfi', '..."
2,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P735', 'name': 'utónév', 'des...","{'wikidata_id': 'Q17498051', 'name': 'József',..."
3,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P570', 'name': 'halálozási id...","{'time': '+1988-02-02T00:00:00Z', 'timezone': ..."
4,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P106', 'name': 'foglalkozás',...","{'wikidata_id': 'Q49757', 'name': 'költő', 'de..."


In [38]:
triplets.source.apply(lambda d: d['wiki_url']).unique()

array(['/wiki/Bencze_J%C3%B3zsef_(k%C3%B6lt%C5%91)',
       '/wiki/Erg_%C3%81goston', '/wiki/Szokolay_Zolt%C3%A1n', ...,
       '/wiki/Bal%C3%A1zs_F._Attila', '/wiki/Babik_J%C3%B3zsef',
       '/wiki/Grecs%C3%B3_Kriszti%C3%A1n'], dtype=object)

In [44]:
res = requests.get('https://hu.wikipedia.org/wiki/Bencze_J%C3%B3zsef_(k%C3%B6lt%C5%91)')

In [46]:
soup = BeautifulSoup(res.text)

In [34]:
def get_wikipedia_url_from_wikidata_id(wikidata_id, lang='en', debug=False):
    

    url = (
        'https://www.wikidata.org/w/api.php'
        '?action=wbgetentities'
        '&props=sitelinks/urls'
        f'&ids={wikidata_id}'
        '&format=json')
    json_response = requests.get(url).json()
    if debug: print(wikidata_id, url, json_response) 

    entities = json_response.get('entities')    
    if entities:
        entity = entities.get(wikidata_id)
        if entity:
            sitelinks = entity.get('sitelinks')
            if sitelinks:
                if lang:
                    # filter only the specified language
                    sitelink = sitelinks.get(f'{lang}wiki')
                    if sitelink:
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            return requests.utils.unquote(wiki_url)
                else:
                    # return all of the urls
                    wiki_urls = {}
                    for key, sitelink in sitelinks.items():
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            wiki_urls[key] = requests.utils.unquote(wiki_url)
                    return wiki_urls
    return None   

In [31]:
get_wikipedia_url_from_wikidata_id('Q466479', lang='hu')

'https://hu.wikipedia.org/wiki/A_helység_kalapácsa'

# TODO
- get poet page contents
- extract sentences with destination entity match (use aliases), mask destination entity in sentence
- create sentence <--> relation database
- train relation classifier